In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install --no-index --no-deps   ../input/textstat/textstat/*.whl  
#! pip install --no-index --no-deps --ignore-installed  ../input/textstat/textstat/*.whl  

In [ ]:
import numpy as np 
import pandas as pd 
import os
import textstat
import matplotlib.pyplot as plt


In [ ]:
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
sub = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")


In [ ]:
train['word_count'] = train['excerpt'].apply(lambda x: len(str(x).split()))
test['word_count'] = test['excerpt'].apply(lambda x: len(str(x).split()))

In [ ]:
import textstat

In [ ]:
def feature_engineering(df):
    df['sentence_count'] = df['excerpt'].apply(lambda x: textstat.sentence_count(x))
    df['syllable_count'] = df['excerpt'].apply(lambda x: textstat.syllable_count(x,lang='en_US'))
    df['word_per_sentence'] = df.apply(lambda row: row.word_count/row.sentence_count, axis=1)
    df['syllable_per_sentence'] = df.apply(lambda row: row.syllable_count/row.sentence_count, axis=1)
    df['syllable_per_word'] = df.apply(lambda row: row.syllable_count/row.word_count, axis=1)

    df['flesch_reading_ease'] = df['excerpt'].apply(lambda x: textstat.flesch_reading_ease(x))
    df['automated_readability_index'] = df['excerpt'].apply(lambda x: textstat.automated_readability_index(x))
    df['linsear_write_formula'] = df['excerpt'].apply(lambda x: textstat.linsear_write_formula(x))
    
    return df

In [ ]:
%%time

train = feature_engineering(train)
test = feature_engineering(test)

In [ ]:
col = list(train.columns)
col.remove('id')
col.remove('url_legal')
col.remove('license')
col.remove('excerpt')
col.remove('standard_error')
col.remove('target')
#col

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression(normalize=True)
lin_reg.fit(train[col],train['target'])

In [ ]:
pred = lin_reg.predict(test[col])

In [ ]:
test['target'] = pred
test.iloc[:, [0,13]]
op_DF = test.iloc[: , [0,13]]

In [ ]:
op_DF.to_csv("submission.csv" , index=False)

In [ ]:
op_DF